In [1]:
!tar -xzf "/content/drive/My Drive/3/1.tar.gz" -C /content

In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from typing import List
from torchvision import models
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import time
import sys
import os

In [3]:
class ecoc_classifier(ClassifierMixin, BaseEstimator):

    def __init__(self, model_constructer=None, ecoc_matrix=None, model_list=None, code_word_length=0):
        '''
        a slightly moddified version of my ecoc model specific to this test

        '''
        self.model_constructer = model_constructer
        self.ecoc_matrix = ecoc_matrix
        self.model_list = model_list
        self.code_word_length = code_word_length

    def Hdistance(self, model_output : List , code_word : List ):# determins hamming distance

        '''
        counts the  diffrance in bits
        '''

        distance = 0
        pos = 0
        while(pos < self.code_word_length):

            if( int(model_output[pos]) !=  code_word[pos] ):

                distance += 1

            pos += 1

        return distance

    def determinLable(self, results):

        '''
        when given an list of output codes from the models, this assigns a list
        of code words from the ecoc matrix which are the smallest hamming distance
        '''

        output = np.empty( ( results.shape[0], self.code_word_length ) )

        item = 0
        while(item < results.shape[0]):

            smallest_distance = -1

            for code_word in self.ecoc_matrix :

                distance = self.Hdistance(results[item], code_word)

                if( distance < smallest_distance or smallest_distance == -1):

                    smallest_distance = distance
                    output_code = code_word

            output[item] = np.array(output_code, copy=True)
            item += 1

        return output

    def fit(self, X, y, **kwargs):

        self.code_word_length = len(self.ecoc_matrix[0])

        '''
        a standerd implementation of fit used by all sklearn models

        in this case it initalzes a model for each column of the ecoc matrix,
        and then calls fit to train it on the bits of the column. after wards the
        model is append to a list for latter use
        '''

        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y

        if(self.model_list == None):

            self.model_list = []

            if(self.model_constructer != None):

                bit_pos = 0
                while(bit_pos < self.code_word_length):

                    self.model_list.append(self.model_constructer())
                    bit_pos += 1

            kwargs['column'] = None

            bit_pos = 0
            while(bit_pos < self.code_word_length):

                kwargs['column'] = bit_pos

                self.model_list[bit_pos].fit(np.load("/content/1/images_{}.npy".format(bit_pos)).swapaxes(1,3).astype(np.float32), np.load("/content/1/lables_{}.npy".format(bit_pos)).astype(np.float32), **kwargs)
                bit_pos += 1

        # Return the classifier
        return self

    def predict(self, X, y=None):

        '''
        a standerd implementation of the predict function used by all sklearn models.

        here after checking if the data is vailid it is feed into each model of the list, and a new output code
        is made from the outputs which is then check against the ecoc matrix to see which row the new code word
        is closest to.
        '''

        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])

        print('predicting')

        results = np.empty((self.code_word_length,) + (X.shape[0],) + (1,))

        pos = 0;
        for model in self.model_list:

            results[pos] = model.predict( X ).cpu().numpy()
            pos += 1

        results = results.reshape((self.code_word_length, X.shape[0])).T.round()

        global output_array
        output_array = results

        return self.determinLable(results)

    def score(self, X, y):
        results = self.predict(X)
        right = 0

        pos = 0
        for sample in results:

            if (sample == y[pos]).all():

                right += 1


            pos += 1

        return right/X.shape[0]

In [4]:
class Main_Model(nn.Module):

    '''
    a pytorch binary classifier 
    '''

    device = None

    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        super(Main_Model,self).__init__()

        eye = models.resnet18(pretrained=True)
        eye.require_grad = False
        eye_out = eye.fc.out_features
        self.prams = nn.ModuleList([eye,
                                    nn.Linear(eye_out,64),
                                    nn.ReLU(),
                                    nn.Linear(64,10),
                                    nn.ReLU(),
                                    nn.Linear(10,1),
                                    ]).to(self.device)
    def forward(self,x):

        for mod in self.prams:
            x = mod(x)
        return x

    def fit(self,X, y, batch_size, epochs, column):
        self.prams.train(True)
        train_set = torch.utils.data.TensorDataset(torch.tensor(X),torch.tensor(y))
        train_set = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False)
        optimizer = torch.optim.Adam(self.prams.parameters(), lr=0.0001)
        lr_schedular = torch.optim.lr_scheduler.StepLR(optimizer, 5)
        loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
        #loss_fn = nn.CrossEntropyLoss()
        pos = 0
        total_time = 0
        while( pos < epochs):
            start_time = time.time()
            running_loss = 0
            for image_batch, label_batch in iter(train_set):
                image_batch, label_batch = image_batch.to(self.device), label_batch.to(self.device)

                optimizer.zero_grad()
                outputs = self(image_batch)
                loss = loss_fn(torch.flatten(outputs),label_batch)
                loss.backward()
                optimizer.step()
                running_loss += loss

            epoch_time = (time.time() - start_time)
            print("-{}/{}".format(pos+1,epochs),"train time:", epoch_time,"loss total:", running_loss.item())
            accuracy = self.eval(test_images, test_lables, 500, column)
            lr_schedular.step()
            total_time += epoch_time

            pos += 1

        global accuracy_array
        accuracy_array.append(accuracy)
        global time_totals
        time_totals.append(total_time)        

    def predict(self,X):

      X = torch.tensor(X, device=self.device)

      with torch.no_grad():
          output = self(X)

          return torch.sigmoid(output).round()

    def eval(self,X, y, batch_size, column):
        self.prams.train(False)
        test_set = torch.utils.data.TensorDataset(torch.tensor(X),torch.tensor(y))
        test_set = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
        right = 0

        for image_batch, label_batch in iter(test_set):
            image_batch, label_batch = image_batch.to(self.device), label_batch.to(self.device)[:,column].unsqueeze(1)
            output_batch = self.predict(image_batch)
            right += (output_batch == label_batch).sum().item()
            
        after = (right/y.shape[0])
        print("after: ", after)
        return after


In [5]:
training_images = []
training_lables = []

general_file_name = "3__1_80_"

test_images = np.load("/content/drive/My Drive/1/{}/test_images.npy".format(general_file_name[3])).swapaxes(1,3).astype(np.float32)
test_lables = np.load("/content/drive/My Drive/1/{}/test_lables.npy".format(general_file_name[3])).astype(np.int64)

ecoc_matrix = np.loadtxt("/content/drive/My Drive/10x10", delimiter=',')


In [6]:

main_model = ecoc_classifier(Main_Model, ecoc_matrix=ecoc_matrix)

batch_size = 64
epochs=10

output_array = []

accuracy_array = []

time_totals = []

average_time = np.vectorize(lambda x : x/epochs)

main_model.fit(training_images, training_lables, batch_size=batch_size, epochs=epochs)

print(main_model.score(test_images, test_lables))

np.save('/content/drive/My Drive/{}_accuracy_array'.format(general_file_name), np.array(accuracy_array))
np.save('/content/drive/My Drive/{}_output_array'.format(general_file_name), output_array)
np.save('/content/drive/My Drive/{}_total_times'.format(general_file_name), np.array(time_totals))
np.save('/content/drive/My Drive/{}_average_times'.format(general_file_name), average_time(time_totals))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



-1/10 train time: 14.950243473052979 loss total: 183.59376525878906


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


after:  0.88535
-2/10 train time: 14.542763233184814 loss total: 141.76185607910156
after:  0.8928
-3/10 train time: 14.60327672958374 loss total: 111.05323791503906
after:  0.9036
-4/10 train time: 14.7224702835083 loss total: 87.03898620605469
after:  0.9126
-5/10 train time: 14.803403377532959 loss total: 74.90347290039062
after:  0.91095
-6/10 train time: 14.908115863800049 loss total: 37.36863327026367
after:  0.9412
-7/10 train time: 14.932056188583374 loss total: 22.147886276245117
after:  0.94535
-8/10 train time: 14.945264101028442 loss total: 12.21973705291748
after:  0.94855
-9/10 train time: 14.96649980545044 loss total: 5.768491744995117
after:  0.9499
-10/10 train time: 15.01505184173584 loss total: 2.5858089923858643
after:  0.9502
-1/10 train time: 15.343553304672241 loss total: 213.77342224121094
after:  0.8658
-2/10 train time: 15.21350884437561 loss total: 161.9730224609375
after:  0.8645
-3/10 train time: 15.245881080627441 loss total: 129.16282653808594
after:  0.8